# Snowflake Data Cleaning

This notebook demonstrates cleaning data from the `SALES_STORE` table in Snowflake by:

1. Reading data from Snowflake.
2. Parsing the `DEMOGRAPHICS` column (XML data).
3. Normalizing the XML data into a structured DataFrame.
4. Saving the cleaned data back to Snowflake.

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, xmlget
from snowflake.connector.pandas_tools import write_pandas
import modin.pandas as pd
import pandas
import snowflake.snowpark.modin.plugin
import xmltodict

## Initialize Snowflake Session

The following cell initializes the Snowflake session. Replace the placeholders with your actual Snowflake credentials.

In [ ]:
def get_session():
    """Initialize Snowflake session."""
    return Session.builder.configs(
        {
            "account": "sfedu02-bob84066",
            "user": "FALCON",
            "password": "Snowflake1",
            "role": "TEAM3_DEVELOPER",
            "warehouse": "TEAM3_WH",
            "database": "TEAM3_DB",
            "schema": "TEAM3_SCHEMA",
        }
    ).create()

# Initialize the session
session = get_session()

## Define XML Parsing Function

The following function parses XML data from the `DEMOGRAPHICS` column into a dictionary, extracting the `StoreSurvey` section.

In [ ]:
def parse_xml(xml_data):
    """Parse the XML data into a dictionary and extract StoreSurvey section."""
    parsed_dict = xmltodict.parse(xml_data)
    store_survey = parsed_dict.get("StoreSurvey", {})
    return store_survey

## Clean `SALES_STORE` Data

This function performs the following steps:

1. Reads the `SALES_STORE` table from Snowflake.
2. Parses the `DEMOGRAPHICS` column using the `parse_xml` function.
3. Normalizes the parsed XML data into a new DataFrame.
4. Merges the parsed data back into the original DataFrame.
5. Drops unnecessary columns.
6. Writes the cleaned data back to Snowflake.

In [ ]:
def clean_sales_store_data(session):
    """Clean the SALES_STORE data and save it back to Snowflake."""
    pd.set_option("display.max_columns", None)

    # Read data from Snowflake
    SalesStoreData = pd.read_snowflake("SALES_STORE")
    SalesStoreData = pd.to_pandas(SalesStoreData)

    # Parse the DEMOGRAPHICS column
    parsed_columns = SalesStoreData["DEMOGRAPHICS"].apply(parse_xml)

    # Create a new DataFrame from the parsed data
    parsed_df = pd.json_normalize(parsed_columns)
    parsed_df = pd.to_pandas(parsed_df)

    # Merge the parsed columns back with the original DataFrame
    SalesStoreDataCleaned = pandas.concat([SalesStoreData, parsed_df], axis=1)
    SalesStoreDataCleaned = SalesStoreDataCleaned.drop(columns=["DEMOGRAPHICS", "@xmlns"])
    SalesStoreDataCleaned = SalesStoreDataCleaned.reset_index(drop=True)

    # Write the cleaned data back to Snowflake
    Session.write_pandas(
        self=session,
        df=SalesStoreDataCleaned,
        table_name="SALES_STORE_CLEANED",
        auto_create_table=True,
        overwrite=True,
    )
    return SalesStoreDataCleaned

## Execute the Cleaning Process

The following cell calls the `clean_sales_store_data` function to clean and save the data back to Snowflake.

In [ ]:
# Clean the data
cleaned_data = clean_sales_store_data(session)

# Display the first few rows of the cleaned data
cleaned_data.head()